In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

Считываем файл с помощью pandas

In [2]:
df = pd.read_csv('heart_disease_dataset.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [3]:
df.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,1
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,1
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1
302,38,1,3,138,175,0,0,173,0,0.0,1,-100000,3,0


Сразу замечаем странное значение CA. 
Получим информацию о данных

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  num       303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [5]:
countNoDisease = len(df[df.num == 0])
countHaveDisease = len(df[df.num == 1])
print(f'Число пациентов без болезней: {countNoDisease}',end=', ')
print(f'Количество пациентов: 303')

Число пациентов без болезней: 164, Количество пациентов: 303


Удалим дубликаты

In [6]:
df = df.drop_duplicates()
df.shape

(303, 14)

Дубликатов не было

In [7]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

Null значения тоже отсутсвуют

In [8]:
df.nunique()

age          41
sex           2
cp            4
trestbps     50
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          4
num           2
dtype: int64

In [9]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.438944,0.679868,3.158416,131.689769,246.693069,0.148515,0.990099,149.607261,0.326733,1.039604,1.600660,-1319.468647,-655.363036,0.458746
std,9.038662,0.467299,0.960126,17.599748,51.776918,0.356198,0.994971,22.875003,0.469794,1.161075,0.616226,11432.566205,8111.366638,0.499120
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,-100000.000000,-100000.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,1.000000


Замечаем странные минимальные значения столбца 'са' и 'thal'

In [10]:
df = df[df['ca'] >= 0]
df = df[df['thal'] >= 0]
df.shape

(297, 14)

In [11]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
count,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000,297.000000
mean,54.542088,0.676768,3.158249,131.693603,247.350168,0.144781,0.996633,149.599327,0.326599,1.055556,1.602694,0.676768,4.730640,0.461279
std,9.049736,0.468500,0.964859,17.762806,51.997583,0.352474,0.994914,22.941562,0.469761,1.166123,0.618187,0.938965,1.938629,0.499340
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000
50%,56.000000,1.000000,3.000000,130.000000,243.000000,0.000000,1.000000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,276.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,1.000000


In [12]:
df.corr()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
age,1.000000,-0.092399,0.110471,0.290476,0.202644,0.132062,0.149917,-0.394563,0.096489,0.197123,0.159405,0.362210,0.126586,0.227075
sex,-0.092399,1.000000,0.008908,-0.066340,-0.198089,0.038850,0.033897,-0.060496,0.143581,0.106567,0.033345,0.091925,0.383652,0.278467
cp,0.110471,0.008908,1.000000,-0.036980,0.072088,-0.057663,0.063905,-0.339308,0.377525,0.203244,0.151079,0.235644,0.268500,0.408945
trestbps,0.290476,-0.066340,-0.036980,1.000000,0.131536,0.180860,0.149242,-0.049108,0.066691,0.191243,0.121172,0.097954,0.138183,0.153490
chol,0.202644,-0.198089,0.072088,0.131536,1.000000,0.012708,0.165046,-0.000075,0.059339,0.038596,-0.009215,0.115945,0.010859,0.080285
fbs,0.132062,0.038850,-0.057663,0.180860,0.012708,1.000000,0.068831,-0.007842,-0.000893,0.008311,0.047819,0.152086,0.062209,0.003167
restecg,0.149917,0.033897,0.063905,0.149242,0.165046,0.068831,1.000000,-0.072290,0.081874,0.113726,0.135141,0.129021,0.018795,0.166343
thalach,-0.394563,-0.060496,-0.339308,-0.049108,-0.000075,-0.007842,-0.072290,1.000000,-0.384368,-0.347640,-0.389307,-0.268727,-0.274831,-0.423817
exang,0.096489,0.143581,0.377525,0.066691,0.059339,-0.000893,0.081874,-0.384368,1.000000,0.289310,0.250572,0.148232,0.326927,0.421355
oldpeak,0.197123,0.106567,0.203244,0.191243,0.038596,0.008311,0.113726,-0.347640,0.289310,1.000000,0.579037,0.294452,0.344976,0.424052


Теперь значения выглядят равномернее. Приготовимся к расчету лучшей метрики

In [13]:
drop_columns = ['num']
features = df.drop(drop_columns, axis=1)
target = df['num']  

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=12345)

ЛОГИЧЕСКАЯ РЕГРЕССИЯ

In [39]:
log_regression = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
log_regression.fit(X_train,y_train)

y_pred = log_regression.predict(X_test)
f1 = f1_score(y_test, y_pred, average="macro")
print(f"f1 = {f1}")
precision = precision_score(y_test, y_pred, average="macro")
print(f"precision = {precision}")
recall = recall_score(y_test, y_pred, average="macro")
print(f"recall = {recall}")
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy = {accuracy}")

f1 = 0.8172844922132046
precision = 0.8318181818181818
recall = 0.8130360205831904
accuracy = 0.8235294117647058


ДЕРЕВО РЕШЕНИЙ

In [41]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="macro")
print(f"f1 = {f1}")
precision = precision_score(y_test, y_pred, average="macro")
print(f"precision = {precision}")
recall = recall_score(y_test, y_pred, average="macro")
print(f"recall = {recall}")
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy = {accuracy}")

f1 = 0.6833513124775261
precision = 0.685072463768116
recall = 0.6825328759291023
accuracy = 0.6890756302521008


KNN

In [43]:
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="macro")
print(f"f1 = {f1}")
precision = precision_score(y_test, y_pred, average="macro")
print(f"precision = {precision}")
recall = recall_score(y_test, y_pred, average="macro")
print(f"recall = {recall}")
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy = {accuracy}")

f1 = 0.49145299145299143
precision = 0.49161931818181814
recall = 0.4915666094911378
accuracy = 0.4957983193277311


SVC

In [44]:
model = SVC(kernel='linear') 
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average="macro")
print(f"f1 = {f1}")
precision = precision_score(y_test, y_pred, average="macro")
print(f"precision = {precision}")
recall = recall_score(y_test, y_pred, average="macro")
print(f"recall = {recall}")
accuracy = accuracy_score(y_test, y_pred)
print(f"accuracy = {accuracy}")

f1 = 0.7918367346938775
precision = 0.803003003003003
recall = 0.7884505431675243
accuracy = 0.7983193277310925




Лучшие показатели у Логической регрессии, все ее показатели больше 0.8. Вторая по результатам SVC модель, показатели которой приближены к 0.8. KNN и показали себя гораздо хуже